You can run this notebook directly on Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DaniAffCH/Vessel-Geometric-Transformers/blob/main/main.ipynb)

In [ ]:
import sys

COLAB_RUNTIME = 'google.colab' in sys.modules
if COLAB_RUNTIME:
    !git clone https://github.com/DaniAffCH/Vessel-Geometric-Transformers.git
    !mv Vessel-Geometric-Transformers/* . 
    !pip install -q -r requirements.txt
else:
    !pip install -q -r requirements.txt
    !pre-commit autoupdate
    !pre-commit install


Loading the configuration

In [ ]:
from src.utils import load_config
import os
from config import DatasetConfig, TrainerConfig, BaselineConfig, GatrConfig

config_path = os.path.join("config","config.yaml")

config = load_config(config_path)
dataset_config: DatasetConfig = config.dataset
trainer_config: TrainerConfig = config.trainer
baseline_config: BaselineConfig = config.baseline
gatr_config: GatrConfig = config.gatr

Loading the dataset

In [ ]:
from src.data import VesselDataModule


data = VesselDataModule(dataset_config)
print(f'Train size: {len(data.train_set)}')
print(f'Validation size: {len(data.val_set)}')
print(f'Test size: {len(data.test_set)}')
print(data.train_set[2])

# TODO: analisi dati preliminare. Classi bilanciate? Statistiche sul dataset? Qualche plot figo.

Testing

In [ ]:
from src.lib import PointGeometricAlgebra, TranslationGeometricAlgebra, ScalarGeometricAlgebra, PlaneGeometricAlgebra

print(data.train_set[0].pos[0])
print(PointGeometricAlgebra.fromElement(data.train_set[0].pos[0].unsqueeze(0)))
print()

print(TranslationGeometricAlgebra.fromElement(data.train_set[1].wss)[:, :8])
print()

print(ScalarGeometricAlgebra.fromElement(data.train_set[0].pressure))
print()

print(PlaneGeometricAlgebra.fromElement(data.train_set[0].face.T))

In [ ]:
from src.trainer import VesselTrainer
from src.models import Gatr

trainer = VesselTrainer(trainer_config)


model = Gatr(gatr_config)
trainer.fit(model, data)

Training Loop

In [ ]:
from src.trainer import VesselTrainer
from src.models import BaselineTransformer

model = BaselineTransformer(baseline_config)

trainer.fit(model, data)

In [ ]:
trainer.test(model, data)

----